In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from datetime import datetime, timedelta
from pylab import rcParams
import pickle
import importlib
import sys
import warnings
import os
import sys
import csv
import glob

In [130]:
#Define Datasource
csv_folder_path = "/Users/elroylo/Documents/GitHub/fyp-2024/data/Features_Micro"
csv_file_pattern = "*.csv"
csv_files = glob.glob(f"{csv_folder_path}/{csv_file_pattern}")
produced_data = pd.read_csv("/Users/elroylo/Documents/GitHub/fyp-2024/data/labeled_data/labeled_index.csv")

#Download features data from local
# Create an empty dictionary to store the tables
csv_tables = {}

# Iterate over the CSV files
for csv_file in csv_files:
    # Extract the label from the CSV file name
    label = csv_file.split("/")[-1].split(".")[0]  # Extracts the file name without extension
    
    # Read the CSV file and store it as a table (DataFrame)
    table = pd.read_csv(csv_file)
    
    # Assign the table to the corresponding label in the dictionary
    csv_tables[label] = table
    
for label,table in csv_tables.items():
    print(f"{label}")
    print(table)

Long-Term_Government_Bond Yields_China
           Date  Value  YOY (%)      index
0    2023-08-01   2.66    -2.56  000001.SS
1    2023-07-01   2.64    -6.71  000001.SS
2    2023-06-01   2.68    -2.90  000001.SS
3    2023-05-01   2.78    -2.11  000001.SS
4    2023-04-01   2.85     2.89  000001.SS
..          ...    ...      ...        ...
255  2002-05-01   2.50      NaN  000001.SS
256  2002-04-01   2.74      NaN  000001.SS
257  2002-03-01   3.12      NaN  000001.SS
258  2002-02-01   3.20      NaN  000001.SS
259  2002-01-01   3.21      NaN  000001.SS

[260 rows x 4 columns]
Recession_Indicators_UK
          Date  YOY (%)  index
0     1/9/2022        0  ^FTLC
1     1/8/2022        0  ^FTLC
2     1/7/2022        0  ^FTLC
3     1/6/2022        0  ^FTLC
4     1/5/2022        0  ^FTLC
..         ...      ...    ...
800   1/1/1956        1  ^FTLC
801  1/12/1955        1  ^FTLC
802  1/11/1955        1  ^FTLC
803  1/10/1955        1  ^FTLC
804   1/9/1955        1  ^FTLC

[805 rows x 3 columns]
I

In [131]:
yfinance_original = ["^GSPC", "^FTLC", "^HSI", "^N225", "000001.SS", "^SSMI", "^BSESN", "^GDAXI", "^FCHI", "^BVSP"]
new_column_name = "index"

csv_US = glob.glob(f"{csv_folder_path}/*US.csv")
csv_UK = glob.glob(f"{csv_folder_path}/*UK.csv")
csv_HK = glob.glob(f"{csv_folder_path}/*HK.csv")
csv_JP = glob.glob(f"{csv_folder_path}/*Japan.csv")
csv_CH = glob.glob(f"{csv_folder_path}/*China.csv")
csv_SW = glob.glob(f"{csv_folder_path}/*Switzerland.csv")
csv_IN = glob.glob(f"{csv_folder_path}/*India.csv")
csv_GER = glob.glob(f"{csv_folder_path}/*Germany.csv")
csv_FRA = glob.glob(f"{csv_folder_path}/*France.csv")
csv_BR = glob.glob(f"{csv_folder_path}/*Brazil.csv")

csv_files = [csv_US, csv_UK, csv_HK, csv_JP, csv_CH, csv_SW, csv_IN, csv_GER, csv_FRA, csv_BR]
n = 0

# Loop through the CSV files and add the new column
for csv_file in csv_files:
    # Loop through each file path in the current list
    for file_path in csv_file:
        # Read the CSV file into a DataFrame
        data = pd.read_csv(file_path)
        
        # Check if the "DATE" or "Date" column exists
        if "DATE" in data.columns:
            # Rename the "DATE" column to "date"
            data.rename(columns={"DATE": "date"}, inplace=True)
        elif "Date" in data.columns:
            # Rename the "Date" column to "date"
            data.rename(columns={"Date": "date"}, inplace=True)
        
        # Check if the "index" column already exists
        if new_column_name not in data.columns:
            # Add the new column to the DataFrame
            data[new_column_name] = yfinance_original[n]
            
            # Save the modified DataFrame back to the CSV file
            data.to_csv(file_path, index=False)
    
    n += 1

In [132]:
# Merge the features by countries - Monthly
# Convert the 'date' column to string type
produced_data['date'] = produced_data['date'].astype(str)
produced_data['Date_modified'] = produced_data['date'].str[:7]

# Create an empty DataFrame to store the merged data
merge_file = produced_data.copy()

# Iterate over each sublist within csv_files
for csv_file_list in csv_files:
    # Iterate over the CSV files within each sublist
    for csv_file in csv_file_list:
        data = pd.read_csv(csv_file)
        data['Date'] = pd.to_datetime(data['Date'])

        # Extract the first 7 characters from the 'Date' column in the current CSV file
        data['Date_modified'] = data['Date'].astype(str).str[:7]
        
        # Get the file name from the CSV file path
        file_name = csv_file.split('/')[-1]
        # Extract the index value from the file name
        new_name = file_name.split('.')[0]
        # Rename the 'YOY (%)' column to the index value
        data.rename(columns={'YOY (%)': new_name}, inplace=True)
        
        # Merge the data based on the specified conditions
        merge_file = pd.merge(merge_file,
                              data[['Date_modified', 'index', new_name]],
                              how='left',
                              left_on=['Date_modified', 'index'],
                              right_on=['Date_modified', 'index'])

# Save the merge_file DataFrame to a CSV file
merge_file.to_csv("/Users/elroylo/Documents/GitHub/fyp-2024/data/merge_file.csv", index=False)

In [133]:
#Merge the Marcoeconomic features - Monthly
Marco_Monthly_csv_folder_path = "/Users/elroylo/Documents/GitHub/fyp-2024/data/Features_Marco_Monthly"
Marco_Monthly_csv_file_pattern = "*.csv"
Marco_Monthly_csv_files = glob.glob(f"{Marco_Monthly_csv_folder_path}/{Marco_Monthly_csv_file_pattern}")

# Iterate over each CSV file in Marco_Monthly_csv_files
for file in Marco_Monthly_csv_files:
    data = pd.read_csv(file)
    data['Date'] = pd.to_datetime(data['Date'])

    # Extract the first 7 characters from the 'Date' column in the current CSV file
    data['Date_modified'] = data['Date'].astype(str).str[:7]
    
    # Get the file name from the CSV file path
    file_name = file.split('/')[-1]
    # Extract the index value from the file name
    new_name = file_name.split('.')[0]
    # Rename the 'YOY (%)' column to the index value
    data.rename(columns={'YOY (%)': new_name}, inplace=True)
    
    # Merge the data based on the specified conditions
    merge_file = pd.merge(merge_file,
                          data[['Date_modified', new_name]],
                          how='left',
                          left_on='Date_modified',
                          right_on='Date_modified')

# Save the merge_file DataFrame to a CSV file
merge_file.to_csv("/Users/elroylo/Documents/GitHub/fyp-2024/data/merge_file.csv", index=False)

In [136]:
#Merge the Marcoeconomic features - Daily
Marco_Daily_csv_folder_path = "/Users/elroylo/Documents/GitHub/fyp-2024/data/Features_Marco_Daily"
Marco_Daily_csv_file_pattern = "*.csv"
Marco_Daily_csv_files = glob.glob(f"{Marco_Daily_csv_folder_path}/{Marco_Daily_csv_file_pattern}")

# Iterate over each CSV file in Marco_Monthly_csv_files
for file in Marco_Daily_csv_files:
    data = pd.read_csv(file)
    data['Date'] = pd.to_datetime(data['Date'])
    # Get the file name from the CSV file path
    file_name = file.split('/')[-1]
    # Extract the index value from the file name
    new_name = file_name.split('.')[0]
    # Rename the 'YOY (%)' column to the index value
    data.rename(columns={'YOY (%)': new_name}, inplace=True)
    
    # Merge the data based on the specified conditions
    merge_file = pd.merge(merge_file,
                          data[['Date', new_name]],
                          how='left',
                          left_on='date',
                          right_on='Date')

# Set all NaN values in the merge_file DataFrame to 0
merge_file.fillna(0, inplace=True)

# Save the merge_file DataFrame to a CSV file
merge_file.to_csv("/Users/elroylo/Documents/GitHub/fyp-2024/data/merge_file.csv", index=False)
print(merge_file)

                price    volume        date  price_change  volume_change  \
0         1199.061035         0  1997-07-02      0.000000       0.000000   
1         1199.061035         0  1997-07-02      0.000000       0.000000   
2         1199.061035         0  1997-07-02      0.000000       0.000000   
3         1199.061035         0  1997-07-02      0.000000       0.000000   
4         1199.061035         0  1997-07-02      0.000000       0.000000   
...               ...       ...         ...           ...            ...   
2012089  10591.980469  26724200  2023-11-02      0.008386       0.137892   
2012090  10579.669922  20698900  2023-11-03     -0.001162      -0.225462   
2012091  10576.750000  20601000  2023-11-06     -0.000276      -0.004730   
2012092  10571.030273  21871900  2023-11-07     -0.000541       0.061691   
2012093  10545.559570         0  2023-11-08     -0.002409       0.000000   

         volatility      index  crash_label  1m_crash_label  3m_crash_label  \
0       